# Importando dependências

In [ ]:
!python -m pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# Importando dados

In [ ]:
df_main = pd.read_csv('data/IM_230626_semNP.csv')
df = pd.read_csv('data/IM_230626_semNP.csv')

# Análise exploratória dos dados

A Análise Exploratória de Dados é um processo fundamental no campo da ciência de dados e envolve a exploração inicial de um conjunto de dados para compreender suas características, padrões e insights preliminares. Ela desempenha um papel crucial na etapa de entendimento do problema no método CRISP-DM.
Envolve atividades como visualização de dados, identificação de tendências, distribuições, relações entre variáveis e detecção de valores atípicos. Essa análise ajuda os cientistas de dados a obter uma compreensão mais profunda dos dados antes de prosseguir para as etapas subsequentes do CRISP-DM, como pré-processamento, modelagem e avaliação.

In [ ]:
df_main.shape

Utilizando o comando shape é possível inferir que os dados antes de serem tratados possuem 57301 linhas e 321 colunas.

In [ ]:
pd.set_option("display.max_rows",10)
list(df_main.columns)

Utilizou-se ```list(df_main.columns)``` para retornar uma lista com todas as colunas do Dataset

In [ ]:
df_main.apply(pd.Series.describe, axis=0)


Utilizando o método 'describe', é viável realizar uma síntese abrangente de estatísticas descritivas relacionadas às colunas numéricas presentes em um DataFrame. Esse procedimento revela informações cruciais, fundamentais para direcionar a etapa preliminar de processamento dos dados.
Ao inspecionar as estatísticas descritivas das colunas, é possível identificar a presença de valores nulos em determinadas categorias. Tais valores em falta podem exercer um impacto significativo sobre a análise subsequente e até mesmo sobre o treinamento de modelos futuros. Portanto, é uma prática recomendável eliminar esses valores ausentes durante a fase de pré-processamento.

In [ ]:
df_main.info()

In [ ]:
df_main.info(verbose=True)

O método ```.info()``` foi utilizado para obter informações do DataFrame. De maneira específica, analisar quais são os tipos de variáveis.
É identificar colunas com variáveis numéricas observando os tipos de dados associados a essas colunas. Valores inteiros e de ponto flutuante são indicativos de variáveis numéricas. Ademais, identificar colunas com variáveis categóricas observando os tipos de dados e a quantidade de valores únicos em cada coluna. Geralmente, os tipos de dados associados a variáveis categóricas são object ou categorical.
Tal processo é importante, pois, posteriormente no pré-processamento de dados variáveis categóricas serão codificadas para uma melhor manipulação.

In [ ]:
df_main.isnull().sum().sort_values(ascending=False).head(10)

Na célula anterior foi calculado as colunas do DataFrame as quais possuem a maior quantidade de valores nulos. Tal análise é de extrema importância pois mostra colunas de dados que não são adequadas de serem usadas, visto a quantidade de valores não existentes. A utilização de colunas com diversos valores nulos pode não ser proveitosa e pode ocasionar inconsistências. Sendo assim, as colunas analisadas acima serão descartadas posteriormente, tendo em vista que, a quantidade de linhas total do DataFrame é 57301.


In [ ]:
forma_condominio_counts = df_main['Forma_Condominio'].value_counts()

sns.set(style="whitegrid")

forma_condominio_counts = df_main['Forma_Condominio'].value_counts()

plt.figure(figsize=(10, 6)) 

bar_plot = sns.barplot(x=forma_condominio_counts.values, y=forma_condominio_counts.index, palette="Blues")

plt.title('Formas de Condomínio de FIDCs')
plt.xlabel('Quantidade')
plt.ylabel('Forma de Condomínio')

for index, value in enumerate(forma_condominio_counts.values):
    bar_plot.text(value, index, str(value))

plt.tight_layout()
plt.show()

Análise da quantidade de fundos que são do tipo de condomínio fechado e de condomínio aberto.
É possível concluir que, a quantidade de fundos de condominio fechado é quase três vezes maior que as de condomínio aberto.
- Condomínio Aberto: Os fundos de investimento abertos são veículos de investimento que permitem a entrada e saída de investidores a qualquer momento. 
- Condomínio Fechado: Os fundos de investimento fechados, por outro lado, têm um número limitado de cotas disponíveis, e essas cotas não são emitidas ou resgatadas com a mesma facilidade que nos fundos abertos.

In [ ]:
df_main['VR_DC_Com_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] + df_main ['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Aquisicao_Provisao_Reducao']
df_main['VR_DC_Sem_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes'] + df_main ['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao']


In [ ]:
tipos_fundo = [
     'FECHADO',
     'ABERTO',
]

colunas_tipo_com_risco = ['Data_Entrega', 'VR_DC_Com_Riscos_E_Beneficios', 'Forma_Condominio']
colunas_tipo_sem_risco = ['Data_Entrega', 'VR_DC_Sem_Riscos_E_Beneficios', 'Forma_Condominio']
filtro = df_main['Forma_Condominio'].isin(tipos_fundo)

In [ ]:
com_risco = df_main.loc[filtro, colunas_tipo_com_risco]
com_risco['Data_Entrega'] = pd.to_datetime(com_risco['Data_Entrega'])

plt.style.use('seaborn-whitegrid')
colors = ['dodgerblue', 'limegreen']
markers = ['o', 'o']

plt.figure(figsize=(12, 8))

for i, forma_condominio in enumerate(tipos_fundo):
    fundo_data = com_risco[com_risco['Forma_Condominio'] == forma_condominio]
    plt.scatter(
        fundo_data['Data_Entrega'], fundo_data['VR_DC_Com_Riscos_E_Beneficios'],
        marker=markers[i], color=colors[i], linewidth=1, edgecolor='black',
        s=100, label=forma_condominio
    )

plt.title('VR com riscos e benefícios dos diferentes tipos de condomínio', fontsize=16, fontweight='bold')
plt.xlabel('Data de Entrega', fontsize=12)
plt.ylabel('VR DC Com Riscos E Beneficios', fontsize=12)
plt.grid(True, alpha=0.5)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.gca().yaxis.grid(True, alpha=0.7)

plt.ylim(bottom=0)

legend = plt.legend()
for handle in legend.legendHandles:
    handle.set_sizes([40])
    
plt.show()

In [ ]:
sem_risco = df_main.loc[filtro, colunas_tipo_sem_risco]
sem_risco['Data_Entrega'] = pd.to_datetime(sem_risco['Data_Entrega'])

plt.style.use('seaborn-whitegrid')
colors = ['dodgerblue', 'limegreen']
markers = ['o', 'o']

plt.figure(figsize=(12, 8))

for i, forma_condominio in enumerate(tipos_fundo):
    fundo_data = sem_risco[sem_risco['Forma_Condominio'] == forma_condominio]
    plt.scatter(
        fundo_data['Data_Entrega'], fundo_data['VR_DC_Sem_Riscos_E_Beneficios'],
        marker=markers[i], color=colors[i], linewidth=1, edgecolor='black',
        s=100, label=forma_condominio 
    )

plt.title('VR sem riscos e benefícios dos diferentes tipos de condomínio', fontsize=16, fontweight='bold')
plt.xlabel('Data de Entrega', fontsize=12)
plt.ylabel('VR DC Sem Riscos E Beneficios', fontsize=12)
plt.grid(True, alpha=0.5)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.gca().yaxis.grid(True, alpha=0.7)

plt.ylim(bottom=0)

legend = plt.legend()
for handle in legend.legendHandles:
    handle.set_sizes([40])

plt.show()

In [ ]:
df_main['VR_DC'] = df_main['VR_DC_Com_Riscos_E_Beneficios'] + df_main ['VR_DC_Sem_Riscos_E_Beneficios']


In [ ]:
colunas_teste = ['Data_Entrega', 'VR_DC', 'Nome_Fundo']


In [ ]:
com_risco = df_main.loc[filtro, colunas_teste]
com_risco['Data_Entrega'] = pd.to_datetime(com_risco['Data_Entrega'])

plt.style.use('seaborn-whitegrid')
colors = ['dodgerblue', 'limegreen']
markers = ['o', 'o']

plt.figure(figsize=(12, 8))

for i, forma_condominio in enumerate(tipos_fundo):
    fundo_data = com_risco[com_risco['Nome_Fundo'] == forma_condominio]
    plt.scatter(
        fundo_data['Data_Entrega'], fundo_data['VR_DC'],
        marker=markers[i], color=colors[i], linewidth=1, edgecolor='black',
        s=100, label=forma_condominio
    )

plt.title('VR com riscos e benefícios dos diferentes tipos de condomínio', fontsize=16, fontweight='bold')
plt.xlabel('Data de Entrega', fontsize=12)
plt.ylabel('VR DC Com Riscos E Beneficios', fontsize=12)
plt.grid(True, alpha=0.5)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.gca().yaxis.grid(True, alpha=0.7)

plt.ylim(bottom=0)

legend = plt.legend()
for handle in legend.legendHandles:
    handle.set_sizes([40])
    
plt.show()

### Verificando todas as colunas mencionadas nas hipóteses sugeridas pela CVM

In [ ]:
relevant_columns = ['Data_Competencia', 'Data_Entrega', 
            'Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes', 'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes', 
            'Ativo_Direitos_Aquisicao_Provisao_Reducao', 'Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes', 
            'Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes', 'Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao',
            'Patrimonio_Liquido', 'Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias',
            'Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil',
            'Negocios_Recompras_Quantidade', 'Negocios_Recompras_Valor', 'Negocios_Recompras_Valor_Contabil', 'VR_DC_Com_Riscos_E_Beneficios', 'VR_DC_Sem_Riscos_E_Beneficios' ]

df_main_hipoteses = df_main[relevant_columns]
missing_percentage = df_main_hipoteses.isnull().sum() / len(df_main)
threshold = 0.47

column_to_drop = missing_percentage[missing_percentage > threshold].index

df_main_hipoteses = df_main_hipoteses.drop(columns=column_to_drop)
df_main_hipoteses.shape

In [ ]:
numeric_columns = df_main_hipoteses.describe().columns
correlation = df_main_hipoteses[numeric_columns].corr()

plt.figure(figsize=(13, 10))
plt.title('Matriz de correlação')
sns.heatmap(correlation, annot=True, fmt='.1f', linewidths='.1', cmap='coolwarm')  # Alterando o cmap para 'coolwarm'
plt.show()

## Target


O Target enviado pelo parceiro consiste em duas análises e fórmulas para avaliar o valor de risco do fundo. O VR (valor de risco), permite verificar se a provisão é suficiente para cobrir os créditos inadimplentes com atraso superior a 30 dias.
Sendo assim, nas próximas células o valor de risco é analisado, tendo em vista que:
Target = (VR dos DCs COM riscos e benefícios + VR dos DCs SEM riscos e benefícios) / Patrimônio Líquido 

In [ ]:
df_main['VR_DC_Com_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] + df_main ['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Aquisicao_Provisao_Reducao']
df_main['VR_DC_Sem_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes'] + df_main ['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao']

df_main['Patrimonio_Liquido'] = df_main['Patrimonio_Liquido'].fillna(df_main['Patrimonio_Liquido'].mean()).replace(to_replace= 0, value=df_main['Patrimonio_Liquido'].mean())
df_main['Target'] = ((df_main['VR_DC_Com_Riscos_E_Beneficios'] + df_main['VR_DC_Sem_Riscos_E_Beneficios']) / df_main['Patrimonio_Liquido'])

df_main = df_main[df_main['Target'] != 0].dropna(subset=['Target'])

df_main['Target'].shape

In [ ]:
df_main['Target']

### Análise de fundos que quebraram

In [ ]:
fundos_problema = [
    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS INDIGO BARTER',
    'CREDIHOME FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',
    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BRAVA CHALLENGE',
    'RUBI FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS MULTISETORIAL',
    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BULLLA',
    'LS INTERBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
    'TURQUESA - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
]

columns_com_risco = ['Data_Entrega', 'VR_DC_Com_Riscos_E_Beneficios', 'Nome_Fundo']
columns_sem_risco = ['Data_Entrega', 'VR_DC_Sem_Riscos_E_Beneficios', 'Nome_Fundo']
filtro = df_main['Nome_Fundo'].isin(fundos_problema)

'Fundos_problema' é uma lista de fundos, que segundo o parceiro, tiveram algum problema ou quebraram.

In [ ]:
com_risco = df_main.loc[filtro, columns_com_risco]

com_risco['Data_Entrega'] = pd.to_datetime(com_risco['Data_Entrega'])

com_risco = com_risco.sort_values(['Nome_Fundo', 'Data_Entrega'])

plt.figure(figsize=(10, 6))

colors = sns.color_palette("rocket", len(fundos_problema))
for i, nome_fundo in enumerate(fundos_problema):
    fundo_data = com_risco[com_risco['Nome_Fundo'] == nome_fundo]
    plt.plot(fundo_data['Data_Entrega'], fundo_data['VR_DC_Com_Riscos_E_Beneficios'], color=colors[i], label=nome_fundo)

plt.title('VR com risco de fundos com problema ao longo do tempo')
plt.xlabel('Data_Entrega')
plt.ylabel('VR DC Com Riscos E Beneficios')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

plt.show()

In [ ]:
sem_risco = df_main.loc[filtro, columns_sem_risco]

sem_risco['Data_Entrega'] = pd.to_datetime(sem_risco['Data_Entrega'])

sem_risco = sem_risco.sort_values(['Nome_Fundo', 'Data_Entrega'])

plt.figure(figsize=(10, 6))

colors = sns.color_palette("rocket", len(fundos_problema))
for i, nome_fundo in enumerate(fundos_problema):
    fundo_data = sem_risco[sem_risco['Nome_Fundo'] == nome_fundo]
    plt.plot(fundo_data['Data_Entrega'], fundo_data['VR_DC_Sem_Riscos_E_Beneficios'], color=colors[i], label=nome_fundo)

plt.title('Valor sem risco de fundos que quebraram')
plt.xlabel('Data_Entrega')
plt.ylabel('VR_DC_Sem_Riscos_E_Beneficios')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
#plt.tight_layout()

plt.show()

In [ ]:
df_main['VR_DC'] = df_main['VR_DC_Com_Riscos_E_Beneficios'] + df_main ['VR_DC_Sem_Riscos_E_Beneficios']


In [ ]:
colunas_teste = ['Data_Entrega', 'VR_DC', 'Carteira']

In [ ]:
sem_risco = df_main.loc[filtro, colunas_teste]

sem_risco['Data_Entrega'] = pd.to_datetime(sem_risco['Data_Entrega'])

sem_risco = sem_risco.sort_values(['Carteira', 'Data_Entrega'])

plt.figure(figsize=(10, 6))

colors = sns.color_palette("rocket", len(fundos_problema))
for i, nome_fundo in enumerate(fundos_problema):
    fundo_data = sem_risco[sem_risco['Carteira'] == nome_fundo]
    plt.plot(fundo_data['Data_Entrega'], fundo_data['VR_DC'], color=colors[i], label=nome_fundo)

plt.title('Valor sem risco de fundos que quebraram')
plt.xlabel('Data_Entrega')
plt.ylabel('VR_DC')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
#plt.tight_layout()

plt.show()

# Pré-Processamento dos dados
O pré-processamento de dados é uma etapa fundamental no processo de análise e modelagem de dados. Ele envolve a limpeza, transformação e organização dos dados brutos em um formato adequado para análise ou para alimentar modelos de machine learning

## Tratamento de Missings 
O tratamento de valores ausentes (missing values) é crucial porque valores faltantes podem levar a conclusões incorretas e prejudicar a qualidade dos resultados da análise de dados e modelagem. O tratamento adequado de missings ajuda a preservar a integridade dos dados, evitando viés e imprecisões, o que é essencial para obter insights confiáveis e tomar decisões informadas.

In [ ]:
limite_percentual = 0.40 
limite_valores_ausentes = len(df_main) * limite_percentual
colunas_para_manter = df_main.columns[df_main.isnull().sum() <= limite_valores_ausentes]
dados_filtrados = df_main[colunas_para_manter]
dados_filtrados

In [ ]:
df_main.isnull().sum().sort_values(ascending = False).head(10)

In [ ]:
null_counts = df_main.isnull().sum().sort_values(ascending=False)

columns_to_drop = null_counts[null_counts > 0].index

df_main.drop(columns=columns_to_drop, inplace=True)

df_main.shape

##### Juntando a tabela Informe Mensal com Informe Mensal Cedentes


In [ ]:
df_classes = pd.read_csv('data/IM_Cedente_230626_semNP.csv')
df_im = pd.read_csv('data/IM_230626_semNP.csv')

df_merge = pd.merge(df_im, df_classes, on='SK_Documento', how='inner')
df_merge.shape

In [ ]:
agg_cols = {'ID_Participante': 'first','ID_Documento': 'first','SK_Documento': 'first','CNPJ': 'first','Data_Competencia': 'first','Data_Entrega': 'first','CNPJ_Administrador': 'first','Nome_Administrador': 'first','Forma_Condominio': 'first','Fundo_Exclusivo': 'first','Cotistas_Vinculados_Interesse': 'first','Prazo_Conversao_Cotas': 'first','Tipo_Prazo_Conversao_Cotas': 'first','Prazo_Pagamento_Resgate': 'first','Tipo_Prazo_Pagamento_Resgate': 'first','Ativo': 'first','Ativo_Disponibilidades': 'first','Ativo_Carteira': 'first','Ativo_Direitos_Aquisicao': 'first','Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes': 'first','Ativo_Direitos_Aquisicao_Creditos_Vencer_Inadimplentes': 'first','Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes': 'first','Ativo_Direitos_Aquisicao_Creditos_Inadimplentes': 'first','Ativo_Direitos_Aquisicao_Creditos_Performar': 'first','Ativo_Direitos_Aquisicao_Creditos_Vencidos_Pendentes': 'first','Ativo_Direitos_Aquisicao_Creditos_Empresas_Recuperacao': 'first','Ativo_Direitos_Aquisicao_Creditos_Receitas_Publicas': 'first','Ativo_Direitos_Aquisicao_Creditos_Acoes_Judiciais': 'first','Ativo_Direitos_Aquisicao_Creditos_Fator_Risco': 'first','Ativo_Direitos_Aquisicao_Creditos_Diversos': 'first','Ativo_Direitos_Aquisicao_Provisao_Reducao': 'first','Ativo_Direitos_Sem_Aquisicao': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Inadimplentes': 'first','Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Performar': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Vencidos_Pendentes': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Empresas_Recuperacao': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Receitas_Publicas': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Acoes_Judiciais': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Fator_Risco': 'first','Ativo_Direitos_Sem_Aquisicao_Creditos_Diversos': 'first','Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao': 'first','Ativo_Valores_Mobiliarios': 'first','Ativo_Debentures': 'first','Ativo_CRI': 'first','Ativo_Notas_Promissorias': 'first','Ativo_Letras Financeiras': 'first','Ativo_Cotas_Fundos_ICVM_555': 'first','Ativo_Outros_Valores_Mobiliarios': 'first','Ativo_Titulos_Federais': 'first','Ativo_CDB': 'first','Ativo_Operacoes_Compromissadas': 'first','Ativo_Outros_Ativos_Renda_Fixa': 'first','Ativo_Cotas_FIDC': 'first','Ativo_Cotas_FIDC_NP': 'first','Ativo_Warrants': 'first','Ativo_Provisao_Debentures_CRI_NP_LF': 'first','Ativo_Provisao_Cotas_FIDC': 'first','Ativo_Provisao_Outros_Ativos': 'first','Ativo_Posicao_Derivativos': 'first','Ativo_Mercado_Termo': 'first','Ativo_Mercado_Opcoes': 'first','Ativo_Mercado_Futuro': 'first','Ativo_Diferencial_Swap': 'first','Ativo_Coberturas_Prestadas': 'first','Ativo_Depositos_Margem': 'first','Ativo_Outros': 'first','Ativo_Curto_Prazo': 'first','Ativo_Longo Prazo': 'first','Carteira': 'first','Carteira_Industrial': 'first','Carteira_Mercado_Imobiliario': 'first','Carteira_Comercial_Total': 'first','Carteira_Comercial': 'first','Carteira_Comercial_Varejo': 'first','Carteira_Arrendamento_Mercantil': 'first','Carteira_Servicos_Total': 'first','Carteira_Servicos': 'first','Carteira_Servicos_Publicos': 'first','Carteira_Servicos_Educacionais': 'first','Carteira_Entretenimento': 'first','Carteira_Agronegocio': 'first','Carteira_Financeiro': 'first','Carteira_Credito Pessoal': 'first','Carteira_Credito_Pessoal_Consignado': 'first','Carteira_Credito_Corporativo': 'first','Carteira_Middle_Market': 'first','Carteira_Veiculos': 'first','Carteira_Imobiliaria_Empresarial': 'first','Carteira_Imobiliaria_Residencial': 'first','Carteira_Outros_Financeiro': 'first','Carteira_Cartao_Credito': 'first','Carteira_Factoring': 'first','Carteira_Factoring_Pessoal': 'first','Carteira_Factoring_Corporativo': 'first','Carteira_Setor_Publico': 'first','Carteira_Precatorios': 'first','Carteira_Creditos_Tributarios': 'first','Carteira_Royalties': 'first','Carteira_Outros_Setor_Publico': 'first','Carteira_Acoes_Judiciais': 'first','Carteira_Propriedade_Intelectual': 'first','Passivo': 'first','Passivo_A_Pagar': 'first','Passivo_Curto_Prazo': 'first','Passivo_Longo_Prazo': 'first','Passivo_Posicao_Derivativos': 'first','Passivo_Mercado_Termo': 'first','Passivo_Mercado_Opcoes': 'first','Passivo_Mercado_Futuro': 'first','Passivo_Diferencial_Swap': 'first','Patrimonio_Liquido': 'first','Patrimonio_Liquido_Medio': 'first','Carteira_Direitos_Aquisicao_Prazo': 'first','Carteira_Direitos_Aquisicao_Prazo_1_30_dias': 'first','Carteira_Direitos_Aquisicao_Prazo_31_60_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_61_90_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_91_120_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_121_150_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_151_180_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_181_360_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_361_720_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias': 'first','Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias': 'first','Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos': 'first','Carteira_Direitos_Aquisicao_Pagos_1_30_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_31_60_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_61_90_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_91_120_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_121_150_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_151_180_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_181_360_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_361_720_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias': 'first','Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias': 'first','Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias': 'first','Negocios_Aquisicoes_Direitos_Aquisicao_Quantidade': 'first','Negocios_Aquisicoes_Direitos_Aquisicao_Valor': 'first','Negocios_Aquisicoes_Direitos_Sem_Aquisicao_Quantidade': 'first','Negocios_Aquisicoes_Direitos_Sem_Aquisicao_Valor': 'first','Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Adimplentes_Quantidade': 'first','Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Adimplentes_Valor': 'first','Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Inadimplentes_Quantidade': 'first','Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Inadimplentes_Valor': 'first','Negocios_Aquisicoes_Direitos_Inadimplentes_Quantidade': 'first','Negocios_Aquisicoes_Direitos_Inadimplentes_Valor': 'first','Negocios_Alienacoes_Cedente_Quantidade': 'first','Negocios_Alienacoes_Cedente_Valor': 'first','Negocios_Alienacoes_Cedente_Valor_Contabil': 'first','Negocios_Alienacoes_Prestadores_Quantidade': 'first','Negocios_Alienacoes_Prestadores_Valor': 'first','Negocios_Alienacoes_Prestadores_Valor_Contabil': 'first','Negocios_Alienacoes_Terceiros_Quantidade': 'first','Negocios_Alienacoes_Terceiros_Valor': 'first','Negocios_Alienacoes_Terceiros_Valor_Contabil': 'first','Negocios_Substituicoes_Quantidade': 'first','Negocios_Substituicoes_Valor': 'first','Negocios_Substituicoes_Valor_Contabil': 'first','Negocios_Recompras_Quantidade': 'first','Negocios_Recompras_Valor': 'first','Negocios_Recompras_Valor_Contabil': 'first','Taxas_Direitos_Aquisicao_Desconto_Compra_Minina': 'first','Taxas_Direitos_Aquisicao_Desconto_Compra_Media_Ponderada': 'first','Taxas_Direitos_Aquisicao_Desconto_Compra_Maxima': 'first','Taxas_Direitos_Aquisicao_Desconto_Venda_Minina': 'first','Taxas_Direitos_Aquisicao_Desconto_Venda_Media_Ponderada': 'first','Taxas_Direitos_Aquisicao_Desconto_Venda_Maxima': 'first','Taxas_Direitos_Aquisicao_Juros_Compra_Minina': 'first','Taxas_Direitos_Aquisicao_Juros_Compra_Media_Ponderada': 'first','Taxas_Direitos_Aquisicao_Juros_Compra_Maxima': 'first','Taxas_Direitos_Aquisicao_Juros_Venda_Minina': 'first','Taxas_Direitos_Aquisicao_Juros_Venda_Media_Ponderada': 'first','Taxas_Direitos_Aquisicao_Juros_Venda_Maxima': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Minina': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Media_Ponderada': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Maxima': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Minina': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Media_Ponderada': 'first','Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Maxima': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Minina': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Media_Ponderada': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Maxima': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Minina': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Media_Ponderada': 'first','Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Maxima': 'first','Taxas_Valores_Mobiliarios_Desconto_Compra_Minina': 'first','Taxas_Valores_Mobiliarios_Desconto_Compra_Media_Ponderada': 'first','Taxas_Valores_Mobiliarios_Desconto_Compra_Maxima': 'first','Taxas_Valores_Mobiliarios_Desconto_Venda_Minina': 'first','Taxas_Valores_Mobiliarios_Desconto_Venda_Media_Ponderada': 'first','Taxas_Valores_Mobiliarios_Desconto_Venda_Maxima': 'first','Taxas_Valores_Mobiliarios_Juros_Compra_Minina': 'first','Taxas_Valores_Mobiliarios_Juros_Compra_Media_Ponderada': 'first','Taxas_Valores_Mobiliarios_Juros_Compra_Maxima': 'first','Taxas_Valores_Mobiliarios_Juros_Venda_Minina': 'first','Taxas_Valores_Mobiliarios_Juros_Venda_Media_Ponderada': 'first','Taxas_Valores_Mobiliarios_Juros_Venda_Maxima': 'first','Taxas_Titulos_Federais_Desconto_Compra_Minina': 'first','Taxas_Titulos_Federais_Desconto_Compra_Media_Ponderada': 'first','Taxas_Titulos_Federais_Desconto_Compra_Maxima': 'first','Taxas_Titulos_Federais_Desconto_Venda_Minina': 'first','Taxas_Titulos_Federais_Desconto_Venda_Media_Ponderada': 'first','Taxas_Titulos_Federais_Desconto_Venda_Maxima': 'first','Taxas_Titulos_Federais_Juros_Compra_Minina': 'first','Taxas_Titulos_Federais_Juros_Compra_Media_Ponderada': 'first','Taxas_Titulos_Federais_Juros_Compra_Maxima': 'first','Taxas_Titulos_Federais_Juros_Venda_Minina': 'first','Taxas_Titulos_Federais_Juros_Venda_Media_Ponderada': 'first','Taxas_Titulos_Federais_Juros_Venda_Maxima': 'first','Taxas_CDB_Desconto_Compra_Minina': 'first','Taxas_CDB_Desconto_Compra_Media_Ponderada': 'first','Taxas_CDB_Desconto_Compra_Maxima': 'first','Taxas_CDB_Desconto_Venda_Minina': 'first','Taxas_CDB_Desconto_Venda_Media_Ponderada': 'first','Taxas_CDB_Desconto_Venda_Maxima': 'first','Taxas_CDB_Juros_Compra_Minina': 'first','Taxas_CDB_Juros_Compra_Media_Ponderada': 'first','Taxas_CDB_Juros_Compra_Maxima': 'first','Taxas_CDB_Juros_Venda_Minina': 'first','Taxas_CDB_Juros_Venda_Media_Ponderada': 'first','Taxas_CDB_Juros_Venda_Maxima': 'first','Taxas_Renda_Fixa_Desconto_Compra_Minina': 'first','Taxas_Renda_Fixa_Desconto_Compra_Media_Ponderada': 'first','Taxas_Renda_Fixa_Desconto_Compra_Maxima': 'first','Taxas_Renda_Fixa_Desconto_Venda_Minina': 'first','Taxas_Renda_Fixa_Desconto_Venda_Media_Ponderada': 'first','Taxas_Renda_Fixa_Desconto_Venda_Maxima': 'first','Taxas_Renda_Fixa_Juros_Compra_Minina': 'first','Taxas_Renda_Fixa_Juros_Compra_Media_Ponderada': 'first','Taxas_Renda_Fixa_Juros_Compra_Maxima': 'first','Taxas_Renda_Fixa_Juros_Venda_Minina': 'first','Taxas_Renda_Fixa_Juros_Venda_Media_Ponderada': 'first','Taxas_Renda_Fixa_Juros_Venda_Maxima': 'first','Numero_Cotistas_Senior_Pessoa_Fisica': 'first','Numero_Cotistas_Senior_Pessoa_Juridica_Nao_Financeira': 'first','Numero_Cotistas_Senior_Banco_Comercial': 'first','Numero_Cotistas_Senior_Corretora_Distribuidora': 'first','Numero_Cotistas_Senior_Pessoa_Juridica_Financeira': 'first','Numero_Cotistas_Senior_Investidor_Nao_Residente': 'first','Numero_Cotistas_Senior_Entidade_Aberta_Previdencia_Complementar': 'first','Numero_Cotistas_Senior_Entidade_Fechada_Previdencia_Complementar': 'first','Numero_Cotistas_Senior_Regime_Proprio_Previdencia_Servidores_Publicos': 'first','Numero_Cotistas_Senior_Sociedade_Seguradora': 'first','Numero_Cotistas_Senior_Sociedade_Capitalizacao': 'first','Numero_Cotistas_Senior_FIC_FIDC': 'first','Numero_Cotistas_Senior_FII': 'first','Numero_Cotistas_Senior_Outros_Fundos': 'first','Numero_Cotistas_Senior_Clube_Investimento': 'first','Numero_Cotistas_Senior_Outros': 'first','Numero_Cotistas_Subordinada_Pessoa_Fisica': 'first','Numero_Cotistas_Subordinada_Pessoa_Juridica_Nao_Financeira': 'first','Numero_Cotistas_Subordinada_Banco_Comercial': 'first','Numero_Cotistas_Subordinada_Corretora_Distribuidora': 'first','Numero_Cotistas_Subordinada_Pessoa_Juridica_Financeira': 'first','Numero_Cotistas_Subordinada_Investidor_Nao_Residente': 'first','Numero_Cotistas_Subordinada_Entidade_Aberta_Previdencia_Complementar': 'first','Numero_Cotistas_Subordinada_Entidade_Fechada_Previdencia_Complementar': 'first','Numero_Cotistas_Subordinada_Regime_Proprio_Previdencia_Servidores_Publicos': 'first','Numero_Cotistas_Subordinada_Sociedade_Seguradora': 'first','Numero_Cotistas_Subordinada_Sociedade_Capitalizacao': 'first','Numero_Cotistas_Subordinada_FIC_FIDC': 'first','Numero_Cotistas_Subordinada_FII': 'first','Numero_Cotistas_Subordinada_Outros_Fundos': 'first','Numero_Cotistas_Subordinada_Clube_Investimento': 'first','Numero_Cotistas_Subordinada_Outros': 'first','Liquidez_Imediata': 'first','Liquidez_Ate_30_Dias': 'first','Liquidez_Ate_60_Dias': 'first','Liquidez_Ate_90_Dias': 'first','Liquidez_Ate_180_Dias': 'first','Liquidez_Ate_360_Dias': 'first','Liquidez_Acima_360_Dias': 'first','Garantias_Valor_Total': 'first','Garantias_Percentual': 'first','Sistema_Origem': 'first','Nome_Fundo': 'first','Numero_Cotistas': 'sum','Quantidade_Cotas': 'sum','Valor_Cota': 'sum','Rentabilidade': 'sum','Valor_Total_Captado': 'sum','Quantidade_Cotas_Emitidas': 'sum','Valor_Total_Resgates': 'sum','Quantidade_Cotas_Resgatadas': 'sum','Valor_A_Pagar': 'sum','Quantidade_Cotas_A_Resgatar': 'sum','Valor_Amortizado_Cota': 'sum','Valor_Total_Amortizacao': 'sum','Desempenho_Esperado': 'sum','Desempenho_Realizado': 'sum'}
result = df_merge.groupby('ID_Documento').agg(agg_cols)

In [ ]:
result.info()

In [ ]:
step_2_pretreat_col_count = len(df_merge.columns)
df_merge['Carteira_Direitos_Aquisicao_Prazo_CP'] = df_merge['Carteira_Direitos_Aquisicao_Prazo_1_30_dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_31_60_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_61_90_Dias']
df_merge['Carteira_Direitos_Aquisicao_Prazo_MP'] = df_merge['Carteira_Direitos_Aquisicao_Prazo_91_120_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_121_150_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_151_180_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_181_360_Dias']
df_merge['Carteira_Direitos_Aquisicao_Prazo_LP'] = df_merge['Carteira_Direitos_Aquisicao_Prazo_361_720_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias']
df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_CP'] = df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias']
df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_MP'] = df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias']
df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_LP'] = df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias']
df_merge['Carteira_Direitos_Aquisicao_Pagos_CP'] = df_merge['Carteira_Direitos_Aquisicao_Pagos_1_30_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_31_60_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_61_90_Dias']
df_merge['Carteira_Direitos_Aquisicao_Pagos_MP'] = df_merge['Carteira_Direitos_Aquisicao_Pagos_91_120_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_121_150_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_151_180_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_181_360_Dias']
df_merge['Carteira_Direitos_Aquisicao_Pagos_LP'] = df_merge['Carteira_Direitos_Aquisicao_Pagos_361_720_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias'] + df_merge['Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_CP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_MP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_LP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_CP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_MP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias']
df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_LP'] = df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias'] + df_merge['Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias']
df_merge['Liquidez_CP'] = df_merge['Liquidez_Ate_30_Dias'] + df_merge['Liquidez_Ate_60_Dias'] + df_merge['Liquidez_Ate_90_Dias']
df_merge['Liquidez_MP'] = df_merge['Liquidez_Ate_180_Dias'] + df_merge['Liquidez_Ate_360_Dias']
df_merge['Liquidez_LP'] = df_merge['Liquidez_Acima_360_Dias']

In [ ]:
agg_cols_to_drop = ['Carteira_Direitos_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias', 'Liquidez_Ate_30_Dias', 'Liquidez_Ate_60_Dias', 'Liquidez_Ate_90_Dias', 'Liquidez_Ate_180_Dias', 'Liquidez_Ate_360_Dias', 'Liquidez_Acima_360_Dias']
df_merge.drop(agg_cols_to_drop, axis=1, inplace=True)
step_2_posttreat_col_count = len(df_merge.columns)
print(f"Antes do tratamento tinhamos {step_2_pretreat_col_count} colunas, agora temos {step_2_posttreat_col_count}.")
print(f"Dropamos {step_2_pretreat_col_count - step_2_posttreat_col_count} colunas")

In [ ]:
step_3_pretreat_col_count = len(df_merge.columns)

nunique = df_merge.nunique()
cols_to_drop = nunique[nunique == 1].index

print(f"Dropando {len(cols_to_drop)} colunas onde todos os valores são iguais: ")
print(cols_to_drop)
df_merge = df_merge.drop(cols_to_drop, axis=1)

print(f"Agora temos {len(df_merge.columns)} colunas")

In [ ]:
useless_cols = ["ID_Participante", "SK_Documento", "Data_Entrega", "Nome_Administrador", "Fundo_Exclusivo", "Cotistas_Vinculados_Interesse", "Prazo_Conversao_Cotas", "Tipo_Prazo_Conversao_Cotas", "Prazo_Pagamento_Resgate", "Tipo_Prazo_Pagamento_Resgate", "Ativo_Direitos_Aquisicao_Creditos_Empresas_Recuperacao", "Ativo_Direitos_Aquisicao_Creditos_Receitas_Publicas", "Ativo_Direitos_Aquisicao_Creditos_Acoes_Judiciais", "Ativo_Direitos_Aquisicao_Creditos_Fator_Risco", "Ativo_Direitos_Aquisicao_Creditos_Diversos", "Ativo_Direitos_Sem_Aquisicao_Creditos_Acoes_Judiciais", "Ativo_Direitos_Sem_Aquisicao_Creditos_Diversos", "Passivo_Posicao_Derivativos", "Passivo_Mercado_Termo", "Passivo_Mercado_Opcoes", "Passivo_Mercado_Futuro", "Passivo_Diferencial_Swap", "Patrimonio_Liquido_Medio", "Negocios_Aquisicoes_Direitos_Aquisicao_Quantidade", "Negocios_Aquisicoes_Direitos_Sem_Aquisicao_Quantidade", "Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Adimplentes_Quantidade", "Negocios_Aquisicoes_Direitos_Vencer_Parcelas_Inadimplentes_Quantidade", "Negocios_Aquisicoes_Direitos_Inadimplentes_Quantidade", "Negocios_Alienacoes_Prestadores_Quantidade", "Negocios_Alienacoes_Terceiros_Quantidade", "Negocios_Substituicoes_Quantidade", "Negocios_Alienacoes_Cedente_Valor_Contabil", "Negocios_Alienacoes_Prestadores_Valor_Contabil", "Negocios_Alienacoes_Terceiros_Valor_Contabil", "Negocios_Substituicoes_Valor_Contabil", "Negocios_Alienacoes_Cedente_Quantidade", "Taxas_Direitos_Aquisicao_Desconto_Compra_Minina", "Taxas_Direitos_Aquisicao_Desconto_Compra_Media_Ponderada", "Taxas_Direitos_Aquisicao_Desconto_Compra_Maxima", "Taxas_Direitos_Aquisicao_Desconto_Venda_Minina", "Taxas_Direitos_Aquisicao_Desconto_Venda_Media_Ponderada", "Taxas_Direitos_Aquisicao_Desconto_Venda_Maxima", "Taxas_Direitos_Aquisicao_Juros_Compra_Minina", "Taxas_Direitos_Aquisicao_Juros_Compra_Media_Ponderada", "Taxas_Direitos_Aquisicao_Juros_Compra_Maxima", "Taxas_Direitos_Aquisicao_Juros_Venda_Minina", "Taxas_Direitos_Aquisicao_Juros_Venda_Media_Ponderada", "Taxas_Direitos_Aquisicao_Juros_Venda_Maxima", "Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Minina", "Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Media_Ponderada", "Taxas_Direitos_Sem_Aquisicao_Desconto_Compra_Maxima", "Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Minina", "Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Media_Ponderada", "Taxas_Direitos_Sem_Aquisicao_Desconto_Venda_Maxima", "Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Minina", "Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Media_Ponderada", "Taxas_Direitos_Sem_Aquisicao_Juros_Compra_Maxima", "Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Minina", "Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Media_Ponderada", "Taxas_Direitos_Sem_Aquisicao_Juros_Venda_Maxima", "Taxas_Valores_Mobiliarios_Desconto_Compra_Minina", "Taxas_Valores_Mobiliarios_Desconto_Compra_Media_Ponderada", "Taxas_Valores_Mobiliarios_Desconto_Compra_Maxima", "Taxas_Valores_Mobiliarios_Desconto_Venda_Minina", "Taxas_Valores_Mobiliarios_Desconto_Venda_Media_Ponderada", "Taxas_Valores_Mobiliarios_Desconto_Venda_Maxima", "Taxas_Valores_Mobiliarios_Juros_Compra_Minina", "Taxas_Valores_Mobiliarios_Juros_Compra_Media_Ponderada", "Taxas_Valores_Mobiliarios_Juros_Compra_Maxima", "Taxas_Valores_Mobiliarios_Juros_Venda_Minina", "Taxas_Valores_Mobiliarios_Juros_Venda_Media_Ponderada", "Taxas_Valores_Mobiliarios_Juros_Venda_Maxima", "Taxas_Titulos_Federais_Desconto_Compra_Minina", "Taxas_Titulos_Federais_Desconto_Compra_Media_Ponderada", "Taxas_Titulos_Federais_Desconto_Compra_Maxima", "Taxas_Titulos_Federais_Desconto_Venda_Minina", "Taxas_Titulos_Federais_Desconto_Venda_Media_Ponderada", "Taxas_Titulos_Federais_Desconto_Venda_Maxima", "Taxas_Titulos_Federais_Juros_Compra_Minina", "Taxas_Titulos_Federais_Juros_Compra_Media_Ponderada", "Taxas_Titulos_Federais_Juros_Compra_Maxima", "Taxas_Titulos_Federais_Juros_Venda_Media_Ponderada", "Taxas_CDB_Desconto_Compra_Minina", "Taxas_CDB_Desconto_Compra_Media_Ponderada", "Taxas_CDB_Desconto_Compra_Maxima", "Taxas_CDB_Juros_Compra_Minina", "Taxas_CDB_Juros_Compra_Media_Ponderada", "Taxas_CDB_Juros_Compra_Maxima", "Taxas_Renda_Fixa_Desconto_Compra_Minina", "Taxas_Renda_Fixa_Desconto_Compra_Media_Ponderada", "Taxas_Renda_Fixa_Desconto_Compra_Maxima", "Taxas_Renda_Fixa_Desconto_Venda_Minina", "Taxas_Renda_Fixa_Desconto_Venda_Media_Ponderada", "Taxas_Renda_Fixa_Desconto_Venda_Maxima", "Taxas_Renda_Fixa_Juros_Compra_Minina", "Taxas_Renda_Fixa_Juros_Compra_Media_Ponderada", "Taxas_Renda_Fixa_Juros_Compra_Maxima", "Taxas_Renda_Fixa_Juros_Venda_Minina", "Taxas_Renda_Fixa_Juros_Venda_Media_Ponderada", "Taxas_Renda_Fixa_Juros_Venda_Maxima", "Numero_Cotistas_Senior_Pessoa_Fisica", "Numero_Cotistas_Senior_Pessoa_Juridica_Nao_Financeira", "Numero_Cotistas_Senior_Banco_Comercial", "Numero_Cotistas_Senior_Corretora_Distribuidora", "Numero_Cotistas_Senior_Pessoa_Juridica_Financeira", "Numero_Cotistas_Senior_Investidor_Nao_Residente", "Numero_Cotistas_Senior_Entidade_Aberta_Previdencia_Complementar", "Numero_Cotistas_Senior_Entidade_Fechada_Previdencia_Complementar", "Numero_Cotistas_Senior_Regime_Proprio_Previdencia_Servidores_Publicos", "Numero_Cotistas_Senior_Sociedade_Seguradora", "Numero_Cotistas_Senior_Sociedade_Capitalizacao", "Numero_Cotistas_Senior_FIC_FIDC", "Numero_Cotistas_Senior_FII", "Numero_Cotistas_Senior_Outros_Fundos", "Numero_Cotistas_Senior_Clube_Investimento", "Numero_Cotistas_Senior_Outros", "Numero_Cotistas_Subordinada_Pessoa_Fisica", "Numero_Cotistas_Subordinada_Pessoa_Juridica_Nao_Financeira", "Numero_Cotistas_Subordinada_Banco_Comercial", "Numero_Cotistas_Subordinada_Corretora_Distribuidora", "Numero_Cotistas_Subordinada_Pessoa_Juridica_Financeira", "Numero_Cotistas_Subordinada_Investidor_Nao_Residente", "Numero_Cotistas_Subordinada_Entidade_Aberta_Previdencia_Complementar", "Numero_Cotistas_Subordinada_Entidade_Fechada_Previdencia_Complementar", "Numero_Cotistas_Subordinada_Regime_Proprio_Previdencia_Servidores_Publicos", "Numero_Cotistas_Subordinada_Sociedade_Seguradora", "Numero_Cotistas_Subordinada_Sociedade_Capitalizacao", "Numero_Cotistas_Subordinada_FIC_FIDC", "Numero_Cotistas_Subordinada_FII", "Numero_Cotistas_Subordinada_Outros_Fundos", "Numero_Cotistas_Subordinada_Clube_Investimento", "Numero_Cotistas_Subordinada_Outros", "Ativo_Debentures", "Ativo_CRI", "Ativo_Notas_Promissorias", "Ativo_Letras Financeiras", "Ativo_Cotas_Fundos_ICVM_555", "Ativo_Outros_Valores_Mobiliarios", "Ativo_Cotas_FIDC_NP", "Ativo_Provisao_Debentures_CRI_NP_LF", "Ativo_Provisao_Cotas_FIDC", "Ativo_Provisao_Outros_Ativos", "Ativo_Curto_Prazo", "Ativo_Longo Prazo"]
cols_sectors = ["Carteira", "Carteira_Industrial", "Carteira_Mercado_Imobiliario", "Carteira_Comercial_Total", "Carteira_Comercial", "Carteira_Comercial_Varejo", "Carteira_Arrendamento_Mercantil", "Carteira_Servicos_Total", "Carteira_Servicos", "Carteira_Servicos_Publicos", "Carteira_Servicos_Educacionais", "Carteira_Entretenimento", "Carteira_Agronegocio", "Carteira_Financeiro", "Carteira_Credito Pessoal", "Carteira_Credito_Pessoal_Consignado", "Carteira_Credito_Corporativo", "Carteira_Middle_Market", "Carteira_Veiculos", "Carteira_Imobiliaria_Empresarial", "Carteira_Imobiliaria_Residencial", "Carteira_Outros_Financeiro", "Carteira_Cartao_Credito", "Carteira_Factoring", "Carteira_Factoring_Pessoal", "Carteira_Factoring_Corporativo", "Carteira_Setor_Publico", "Carteira_Precatorios", "Carteira_Outros_Setor_Publico", "Carteira_Acoes_Judiciais", "Carteira_Propriedade_Intelectual"]

df_merge = df_merge.drop(useless_cols, axis=1)
df_merge = df_merge.drop(cols_sectors, axis=1)

print(f"Temos {len(useless_cols) + len(cols_sectors)} colunas")
print(f"Agora temos {len(df_merge.columns)} colunas")

In [ ]:
missing_columns = [col for col in ativos_drop if col not in df_merge.columns]
if missing_columns:
    print("As seguintes colunas não foram encontradas no DataFrame:")
    for col in missing_columns:
        print(col)
else:
    df_merge = df_merge.drop(ativos_drop, axis=1)
    print(f"Tiramos {len(ativos_drop)} colunas")
    print(f"Agora temos {len(df_merge.columns)} colunas")

In [ ]:
for col in df_merge.columns:
    print(col)

## Codificação de variáveis categóricas

A codificação de variáveis categóricas é o processo de transformar valores categóricos, como nomes ou rótulos, em uma forma numérica que possa ser utilizada por algoritmos de aprendizado de máquina. Isso é necessário porque muitos algoritmos só conseguem lidar com dados numéricos. Existem diferentes técnicas de codificação, como codificação one-hot, codificação ordinal e codificação de frequência, cada uma com seus próprios métodos para representar variáveis categóricas em formato numérico. Isso permite que os modelos de aprendizado de máquina utilizem essas variáveis categóricas como entrada para fazer previsões ou análises.

In [ ]:
df_cats = df_merge.select_dtypes(include='object')
df_cats = df_merge.dropna(subset=['Tipo_Prazo_Conversao_Cotas',	'Tipo_Prazo_Pagamento_Resgate'])
df_cats

In [ ]:
df_merge = pd.get_dummies(df_merge, columns=['Forma_Condominio',	'Fundo_Exclusivo',	'Cotistas_Vinculados_Interesse',	'Tipo_Prazo_Conversao_Cotas',	'Tipo_Prazo_Pagamento_Resgate'])
df_merge

# Hipótese

## Hipótese 3
Fundos com substituições de créditos e VR

In [ ]:
df_h3 = df.copy()

In [ ]:
fundos_com_substituicoes_de_credito = ['Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil']

df_h3[fundos_com_substituicoes_de_credito].query('Negocios_Substituicoes_Quantidade > 0')

In [ ]:
df_h3 = df[['Data_Entrega', 'Nome_Fundo', 'Negocios_Substituicoes_Quantidade', 'Negocios_Substituicoes_Valor', 'Negocios_Substituicoes_Valor_Contabil']].query('Negocios_Substituicoes_Quantidade > 0')
display(df_h3.head(5))
display(df_h3.describe())

### Selecionando os fundos com substituições de crédito

In [ ]:
df_h3['VR_DC_Com_Riscos_E_Beneficios'] = df['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] + df ['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - df['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - df['Ativo_Direitos_Aquisicao_Provisao_Reducao']
df_h3['VR_DC_Sem_Riscos_E_Beneficios'] = df['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes'] + df ['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - df['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao']

df_h3_sorted = df_h3.sort_values(by='Negocios_Substituicoes_Valor', ascending=False)

In [ ]:
df_h3_sorted['Nome_Fundo'].unique()

In [ ]:
for fundo in df_h3_sorted['Nome_Fundo'].unique():
    data = df_h3.query(f'Nome_Fundo ==  "{fundo}"')['Negocios_Substituicoes_Valor']
    values = df_h3.query(f'Nome_Fundo ==  "{fundo}"')['VR_DC_Com_Riscos_E_Beneficios']
    plt.scatter(data, values)
    plt.title(f'Renegociação de Crédito x VR Com Riscos E Beneficios')
    plt.xlabel('Substituição de Crédito')
    plt.ylabel('VR_DC_Com_Riscos_E_Beneficios')
    plt.xticks([])
plt.show()

for fundo in df_h3_sorted['Nome_Fundo'].unique():
    data = df_h3.query(f'Nome_Fundo ==  "{fundo}"')['Negocios_Substituicoes_Valor']
    values = df_h3.query(f'Nome_Fundo ==  "{fundo}"')['VR_DC_Sem_Riscos_E_Beneficios']
    plt.scatter(data, values)
    plt.title(f'Renegociação de Crédito x VR Sem Riscos E Beneficios')
    plt.xlabel('Substituição de Crédito')
    plt.ylabel('VR_DC_Sem_Riscos_E_Beneficios')
    plt.xticks([])
plt.show()


Foi possível notar que, VR com riscos e benefícios de fundos com maior substituição de crédito não possuem um padrão: alguns tendem para o negativo, outrs tendem para positivo, e outros são positivos. 
Analisando também os fundos com menor substitução de crédito é notórico que eles, são em sua maioria, dados nulos e pouco úteis para a análise da hipótese. 

Já no caso dos VR sem riscos e benefícios de fundos com maior substituição de crédito sua maioria tem vr sem risco e beneficios nula, alguns outros com no quadrante positivo mas sem um padrão específico. E o dos com menor substitução de crédito são em sua maioria nulos.

## Hipótese 4
FIDCs com renegociações de créditos são mais propensos a apresentar aumento futuro de VR? Qual o nível de renegociações antecede um aumento do VR? As renegociações são identificadas por um aumento de créditos a vencer (campos I.2a.1 e I.2b.1) e diminuição dos valores em atraso (campos I.2a.2.1 e I.2b.2.1 ou V.b e VI.b) no mesmo mês, sem a ocorrência de recompra nem substituição de créditos.

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df_h4 = df_main

df_h4['VR_DC_Com_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] + df_main['Ativo_Direitos_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Aquisicao_Provisao_Reducao']
df_h4['VR_DC_Sem_Riscos_E_Beneficios'] = df_main['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes'] + df_main['Ativo_Direitos_Sem_Aquisicao_Creditos_Inadimplentes'] - df_main['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] - df_main['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao']

df_h4 = df_h4.loc[(df_h4['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes'] != 0) & (df_h4['Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes'] != 0) & (df_h4['Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes'] != 0) & (df_h4['Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes'] != 0) & (df_h4['Carteira_Direitos_Aquisicao_Inadimplentes'] != 0) & (df_h4['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'] != 0)]

num_columns_h4 = ['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes', 'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes', 
                 'Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes', 'Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes', 
                 'Carteira_Direitos_Aquisicao_Inadimplentes', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes',
                 'VR_DC_Com_Riscos_E_Beneficios',	'VR_DC_Sem_Riscos_E_Beneficios']

df_h4 = df_h4[['Nome_Fundo', 'Data_Entrega','VR_DC_Com_Riscos_E_Beneficios','VR_DC_Sem_Riscos_E_Beneficios','Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes', 'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes', 
                 'Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes', 'Ativo_Direitos_Sem_Aquisicao_Parcelas_Inadimplentes', 
                 'Carteira_Direitos_Aquisicao_Inadimplentes', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes']]

df_h4[num_columns_h4] = scaler.fit_transform(df_h4[num_columns_h4])

df_h4_sorted = df_h4.sort_values(by='Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes' ,ascending=False)

display(df_h4_sorted.head(3))
display(df_h4_sorted['Nome_Fundo'].unique())

In [ ]:
for fundo in df_h4_sorted['Nome_Fundo'].unique():
    data = df_h4.query(f'Nome_Fundo ==  "{fundo}"')['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes']
    values = df_h4.query(f'Nome_Fundo ==  "{fundo}"')['VR_DC_Com_Riscos_E_Beneficios']
    plt.scatter(data, values)
    plt.title(f'Substituição de Crédito x VR Com Riscos e Benefícios')
    plt.xlabel('Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes')
    plt.ylabel('VR_DC_Com_Riscos_E_Beneficios')
    plt.xticks([])
plt.show()

for fundo in df_h4_sorted['Nome_Fundo'].unique():
    data = df_h4.query(f'Nome_Fundo ==  "{fundo}"')['Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes']
    values = df_h4.query(f'Nome_Fundo ==  "{fundo}"')['VR_DC_Sem_Riscos_E_Beneficios']
    plt.scatter(data, values)
    plt.title(f'Substituição de Crédito x VR Sem Riscos e Benefícios')
    plt.xlabel('Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes')
    plt.ylabel('VR_DC_Sem_Riscos_E_Beneficios')
    plt.xticks([])
plt.show()

## Hipótese 6
Comportamento dos Valores de Risco quanto a evolução de inadimplência.

In [ ]:
unique_nome_fundo = df_main['Nome_Fundo'].unique()

grouped = df_main.groupby('Nome_Fundo')
result_df = grouped[['VR_DC_Com_Riscos_E_Beneficios', 'VR_DC_Sem_Riscos_E_Beneficios']].sum()

result_df

In [ ]:
result_df.reset_index(inplace=True)

In [ ]:
fig, ax = plt.subplots()

bar_width = 0.35
bar_positions = range(len(result_df))
vr_risco_bars = ax.bar(bar_positions, result_df['VR_DC_Sem_Riscos_E_Beneficios'], bar_width, label='VR_DC_Com_Riscos_E_Beneficios')
vr_s_risco_bars = ax.bar([pos + bar_width for pos in bar_positions], result_df['VR_DC_Sem_Riscos_E_Beneficios'], bar_width, label='VR_DC_Com_Riscos_E_Beneficios')

ax.set_xticks([pos + bar_width / 2 for pos in bar_positions])
ax.set_xticklabels(df_main['Nome_Fundo'])
ax.tick_params(axis='x', which='both', bottom=False)

ax.set_title("Valores de vr_risco e vr_s_risco por Nome_Fundo")
ax.legend()

plt.show()

In [ ]:
df_main['VR_DC'] = df_main['VR_DC_Com_Riscos_E_Beneficios'] + df_main ['VR_DC_Sem_Riscos_E_Beneficios']


In [ ]:
com_risco = df_main.loc[filtro, columns_com_risco]

com_risco['Data_Entrega'] = pd.to_datetime(com_risco['Data_Entrega'])

com_risco = com_risco.sort_values(['Nome_Fundo', 'Data_Entrega'])

plt.figure(figsize=(10, 6))

colors = sns.color_palette("rocket", len(fundos_problema))
for i, nome_fundo in enumerate(fundos_problema):
    fundo_data = com_risco[com_risco['Nome_Fundo'] == nome_fundo]
    plt.plot(fundo_data['Data_Entrega'], fundo_data['VR_DC'], color=colors[i], label=nome_fundo)

plt.title('VR com risco de fundos com problema ao longo do tempo')
plt.xlabel('Data_Entrega')
plt.ylabel('VR DC Com Riscos E Beneficios')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

plt.show()